In [2]:
1+1

2

In [11]:
# STEP 1: Install dependencies
import os
import cv2
import glob
import shutil
import pandas as pd
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Original dataset paths
IMAGE_DIR = "dataset/images"
ANNOTATION_DIR = "dataset/annotations"

# YOLO formatted dataset folder
YOLO_DATASET_DIR = "dataset_yolo"
IMG_OUT_DIR = os.path.join(YOLO_DATASET_DIR, "images")
LBL_OUT_DIR = os.path.join(YOLO_DATASET_DIR, "labels")

# Create folders
for split in ["train", "val"]:
    os.makedirs(os.path.join(IMG_OUT_DIR, split), exist_ok=True)
    os.makedirs(os.path.join(LBL_OUT_DIR, split), exist_ok=True)

# Function to parse XML and save as YOLO format
def convert_xml_to_yolo(xml_file, image_shape):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    yolo_labels = []
    img_w = int(image_shape[1])
    img_h = int(image_shape[0])

    for obj in root.findall("object"):
        cls = obj.find("name").text.strip()
        # Only consider license plates
        if cls.lower() not in ["number_plate", "license_plate", "plate"]:
            continue

        xmlbox = obj.find("bndbox")
        xmin = int(xmlbox.find("xmin").text)
        ymin = int(xmlbox.find("ymin").text)
        xmax = int(xmlbox.find("xmax").text)
        ymax = int(xmlbox.find("ymax").text)

        # YOLO format: class x_center y_center width height (normalized)
        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h

        yolo_labels.append(f"0 {x_center} {y_center} {width} {height}")

    return yolo_labels

# Collect all images and annotations
all_images = glob.glob(os.path.join(IMAGE_DIR, "*.jpg")) + glob.glob(os.path.join(IMAGE_DIR, "*.png"))
train_imgs, val_imgs = train_test_split(all_images, test_size=0.2, random_state=42)

def process_and_save(images, split):
    for img_path in images:
        fname = os.path.basename(img_path)
        name, ext = os.path.splitext(fname)
        xml_path = os.path.join(ANNOTATION_DIR, f"{name}.xml")

        if not os.path.exists(xml_path):
            continue

        # Read image
        img = cv2.imread(img_path)
        h, w = img.shape[:2]

        # Convert annotation to YOLO
        yolo_labels = convert_xml_to_yolo(xml_path, (h, w))

        # Save image
        out_img_path = os.path.join(IMG_OUT_DIR, split, fname)
        shutil.copy(img_path, out_img_path)

        # Save label file
        out_lbl_path = os.path.join(LBL_OUT_DIR, split, f"{name}.txt")
        with open(out_lbl_path, "w") as f:
            f.write("\n".join(yolo_labels))

# Process training and validation sets
process_and_save(train_imgs, "train")
process_and_save(val_imgs, "val")

print("✅ Dataset converted to YOLO format!")


In [ ]:
yaml_content = f"""
train: {os.path.abspath(os.path.join(IMG_OUT_DIR, 'train'))}
val: {os.path.abspath(os.path.join(IMG_OUT_DIR, 'val'))}

nc: 1
names: ['number_plate']
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)

print("✅ data.yaml file created!")


In [ ]:
from ultralytics import YOLO

# Load YOLOv10n (nano model - fast & light)
model = YOLO("yolov8n.pt")  # ultralytics does not yet have yolov10 official, yolov8 works similarly

# Train
model.train(
    data="data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name="car_plate_model"
)


In [ ]:
# Save final weights
trained_model_path = "model.pt"
model.save(trained_model_path)

print(f"✅ Model saved as {trained_model_path}")